In [2]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import os
#import tflearn
#import tensorflow as tf
import random
os.chdir(r"F:\training\Projects\contexual_chatbot\chatbot1")

In [3]:
import json
with open('intents.json') as json_data:
    intents=json.load(json_data)
intents
#8660063617

{u'intents': [{u'context_set': u'',
   u'patterns': [u'Hi',
    u'How are you',
    u'Is anyone there?',
    u'Hello',
    u'Good day'],
   u'responses': [u'Hello, thanks for visiting',
    u'Good to see you again',
    u'Hi there, how can I help?'],
   u'tag': u'greeting'},
  {u'patterns': [u'Bye', u'See you later', u'Goodbye'],
   u'responses': [u'See you later, thanks for visiting',
    u'Have a nice day',
    u'Bye! Come back again soon.'],
   u'tag': u'goodbye'},
  {u'patterns': [u'Thanks', u'Thank you', u"That's helpful"],
   u'responses': [u'Happy to help!', u'Any time!', u'My pleasure'],
   u'tag': u'thanks'},
  {u'patterns': [u'What hours are you open?',
    u'What are your hours?',
    u'When are you open?'],
   u'responses': [u"We're open every day 9am-9pm",
    u'Our hours are 9am-9pm every day'],
   u'tag': u'hours'},
  {u'patterns': [u'Which mopeds do you have?',
    u'What kinds of mopeds are there?',
    u'What do you rent?'],
   u'responses': [u'We rent Yamaha, Piaggio

In [4]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

#print words
#print documents
#print classes
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

(27, 'documents')
(9, 'classes', [u'goodbye', u'greeting', u'hours', u'mopeds', u'opentoday', u'payments', u'rental', u'thanks', u'today'])
(48, 'unique stemmed words', [u"'d", u"'s", u'a', u'acceiv', u'anyon', u'ar', u'bye', u'can', u'card', u'cash', u'credit', u'day', u'do', u'doe', u'good', u'goodby', u'hav', u'hello', u'help', u'hi', u'hour', u'how', u'i', u'is', u'kind', u'lat', u'lik', u'mastercard', u'mop', u'of', u'on', u'op', u'rent', u'see', u'tak', u'thank', u'that', u'ther', u'thi', u'to', u'today', u'we', u'what', u'when', u'which', u'work', u'yo', u'you'])


In [5]:
documents

[([u'Hi'], u'greeting'),
 ([u'How', u'are', u'you'], u'greeting'),
 ([u'Is', u'anyone', u'there', u'?'], u'greeting'),
 ([u'Hello'], u'greeting'),
 ([u'Good', u'day'], u'greeting'),
 ([u'Bye'], u'goodbye'),
 ([u'See', u'you', u'later'], u'goodbye'),
 ([u'Goodbye'], u'goodbye'),
 ([u'Thanks'], u'thanks'),
 ([u'Thank', u'you'], u'thanks'),
 ([u'That', u"'s", u'helpful'], u'thanks'),
 ([u'What', u'hours', u'are', u'you', u'open', u'?'], u'hours'),
 ([u'What', u'are', u'your', u'hours', u'?'], u'hours'),
 ([u'When', u'are', u'you', u'open', u'?'], u'hours'),
 ([u'Which', u'mopeds', u'do', u'you', u'have', u'?'], u'mopeds'),
 ([u'What', u'kinds', u'of', u'mopeds', u'are', u'there', u'?'], u'mopeds'),
 ([u'What', u'do', u'you', u'rent', u'?'], u'mopeds'),
 ([u'Do', u'you', u'take', u'credit', u'cards', u'?'], u'payments'),
 ([u'Do', u'you', u'accept', u'Mastercard', u'?'], u'payments'),
 ([u'Are', u'you', u'cash', u'only', u'?'], u'payments'),
 ([u'Are', u'you', u'open', u'today', u'?'], u'o

In [7]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
X_train = list(training[:,0])
Y_train = list(training[:,1])
Y_train

[[1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0]]

In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
mlp=MLPClassifier(hidden_layer_sizes=(20,20))
mlp.fit(X_train,Y_train)
mlp.predict(X_train[10])

C:\Anaconda\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([[0, 0, 0, 0, 0, 0, 1, 0, 0]])

In [13]:
classes
Y_train

[[1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0]]

In [14]:
X_train[10]

[1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]